In [1]:
import os
import typing
from sklearn.gaussian_process.kernels import *
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

from autograd import grad 
import tensorflow as tf
from keras.losses import categorical_crossentropy, mean_squared_error
from keras import layers
from keras.layers import *
from keras.metrics import AUC
from keras.callbacks import EarlyStopping
from keras.activations import *

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time

import os
import typing
from sklearn.gaussian_process.kernels import *
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from matplotlib import cm
import gpytorch
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
import json
from sklearn.preprocessing import StandardScaler

In [2]:
# Cost function constants
COST_W_UNDERPREDICT = 25.0
COST_W_NORMAL = 1.0
COST_W_OVERPREDICT = 10.0
def cost_function(ground_truth: np.ndarray, predictions: np.ndarray) -> float:
    # Unweighted cost
    cost = (ground_truth - predictions) ** 2
    weights = np.ones_like(cost) * COST_W_NORMAL

    # Case i): underprediction
    mask_1 = predictions < ground_truth
    weights[mask_1] = COST_W_UNDERPREDICT

    # Case ii): significant overprediction
    mask_2 = (predictions >= 1.2*ground_truth)
    weights[mask_2] = COST_W_OVERPREDICT

    # Weigh the cost and return the average
    return np.mean(cost * weights)


In [3]:
# Load the training dateset and test features
x = np.loadtxt('train_x.csv', delimiter=',', skiprows=1)
testx = np.loadtxt('test_x.csv', delimiter=',', skiprows=1)
y = np.loadtxt('train_y.csv', delimiter=',', skiprows=1)
largest = y.max()
largest = 1
print(largest)
y/=largest


1


In [4]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)



In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.tensor(x, dtype=torch.float32).to(device)
# testx = torch.tensor(testx, dtype=torch.float32).to(device)
testx = torch.tensor(testx, dtype=torch.float32).to("cpu")
y = torch.tensor(y, dtype=torch.float32).to(device)




# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x, y, likelihood)


# initialize lengthscale, outputscale and noise to random values between 0 and 0.1
# model.covar_module.base_kernel.lengthscale = np.random.rand(1)*0.1
# model.covar_module.outputscale = np.random.rand(1)*0.1
# likelihood.noise = np.random.rand(1)*0.01




likelihood2 = gpytorch.likelihoods.GaussianLikelihood()
likelihood2.eval()
model2 = ExactGPModel(x, y, likelihood2)
model2 = model2.to('cpu')
model2.eval()


# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# load the model
# model.load_state_dict(torch.load('model_state.pth'))
# model.load_state_dict(torch.load('model_state_improved6.pth'))


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

In [7]:
def test(gp_mean,gp_std):
    params = {'gp_mean': gp_mean.tolist(), 'gp_std': gp_std.tolist()}
    with open('predictions.json', 'w') as f:
        json.dump(params, f)
    # o = os.popen('docker build --tag task1 .').read()
    o = os.popen('docker run --rm -v "%cd%:/results" task1').read()
    o = o.replace("Congratulations, you have passed the checks on public dataset. Your cost is ","").replace("Dumped check file to /results/results_check.byte","").replace("\n","")
    if "improve" in o:
        return 99999
    else:
        return float(o)

In [8]:
model.train()
likelihood.train()
model = model.to(device)

training_iter = 5000
best = 9999
for i in range(training_iter):

    # Zero backpropped gradients from previous iteration
    optimizer.zero_grad()
    # Get output from model
    output = model(x)
    # Calc loss and backprop gradients
    loss = -mll(output, y)
    loss.backward()

    if (best < 21 and i % 25 == 0) or (i % 100 == 0):
        print('Iter %d/%d - Loss: %.3f - Lengthscale: %.3f - Noise: %.3f - Outputscale: %.3f' % (
            i, training_iter, loss.item(),
            model.covar_module.base_kernel.lengthscale.item(),
            model.likelihood.noise.item(),
            model.covar_module.outputscale.item()
        ))
        with torch.no_grad(), gpytorch.settings.fast_pred_var():
            # print("eval")
            torch.save(model.state_dict(), 'model_state_temp.pth')
            model2.load_state_dict(torch.load('model_state_temp.pth'))
            output = model2(testx.to('cpu'))
            gp_mean = output.mean
            gp_mean*=largest
            gp_std = output.stddev
            score = test(gp_mean,gp_std)
            print('Iter %d/%d - Score: %.3f' % (i, training_iter, score))
            if score < best:
                best = score
                torch.save(model.state_dict(), 'model_state_improved6.pth')
                print('Best model saved with score: %.3f' % (best))



    optimizer.step()

Iter 0/5000 - Loss: 171.468 - Lengthscale: 0.693 - Noise: 0.693 - Outputscale: 0.693


c:\ProgramData\Anaconda3\lib\site-packages\gpytorch\distributions\multivariate_normal.py:259: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


Iter 0/5000 - Score: 99999.000
Iter 100/5000 - Loss: 3.763 - Lengthscale: 0.021 - Noise: 2.571 - Outputscale: 5.871
Iter 100/5000 - Score: 52.180
Best model saved with score: 52.180
Iter 200/5000 - Loss: 3.154 - Lengthscale: 0.018 - Noise: 2.603 - Outputscale: 7.808
Iter 200/5000 - Score: 34.516
Best model saved with score: 34.516
Iter 300/5000 - Loss: 2.917 - Lengthscale: 0.017 - Noise: 2.622 - Outputscale: 9.258
Iter 300/5000 - Score: 27.379
Best model saved with score: 27.379
Iter 400/5000 - Loss: 2.805 - Lengthscale: 0.017 - Noise: 2.633 - Outputscale: 10.508
Iter 400/5000 - Score: 25.439
Best model saved with score: 25.439
Iter 500/5000 - Loss: 2.737 - Lengthscale: 0.016 - Noise: 2.637 - Outputscale: 11.657
Iter 500/5000 - Score: 23.180
Best model saved with score: 23.180
Iter 600/5000 - Loss: 2.688 - Lengthscale: 0.016 - Noise: 2.636 - Outputscale: 12.741
Iter 600/5000 - Score: 22.459
Best model saved with score: 22.459
Iter 700/5000 - Loss: 2.647 - Lengthscale: 0.016 - Noise: 2.

In [ ]:
# save the model
# torch.save(model.state_dict(), 'model_state_improved6.pth')

In [ ]:
# # Set into eval mode
# model = model.to('cpu')
# # model.load_state_dict(torch.load('model_state.pth', map_location = torch.device('cpu')))
# model.eval()
# likelihood.eval()
# tt = torch.tensor(X_test, dtype=torch.float32)

# with torch.no_grad(), gpytorch.settings.fast_pred_var():
#     output:MultivariateNormal = model(tt)
#     observed_pred = likelihood(output)
#     gp_mean = observed_pred.mean.cpu().numpy()
#     gp_std = np.sqrt(observed_pred.variance.cpu().detach().numpy())
#     print(output.mean.numpy())
#     print(gp_mean)
#     print(output.stddev.numpy())
#     print(gp_std)

# for mult in np.arange(0.32, 0.5, 0.02):
#     for tresh in np.arange(1.01, 1.10, 0.01):
#         predictions = gp_mean+gp_std*mult
#         for i in range(gp_mean.size):
#             tresh = tresh
#             if predictions[i]>tresh*gp_mean[i]:
#                 predictions[i] = tresh*gp_mean[i]

#         print("Mult: ", mult, "Tresh: ", tresh, "Cost: ", cost_function(y_test, predictions))

